In [1]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import PIL
import io
import html
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [2]:
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [3]:
!pip install DeepFace

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 86.5 MB/s eta 0:00:00


In [4]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
from deepface import DeepFace

25-11-16 20:02:55 - Directory /root/.deepface has been created
25-11-16 20:02:55 - Directory /root/.deepface/weights has been created


In [11]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import io
from base64 import b64encode

def text_to_bytes(bbox_array, text="Vision Programmer", text_position=(150, 275), font_size=50):
    """
    Overlay specified text on a given RGBA image array and return its Base64-encoded string.

    Params:
        bbox_array (numpy.ndarray): A NumPy array of shape (480, 640, 4) representing an RGBA image.
        text (str): The text to draw on the image.
        text_position (tuple): The position of the text as (x, y).
        font_size (int): The font size of the text.

    Returns:
        str: Base64-encoded string of the image.
    """
    # Ensure bbox_array is a valid RGBA image
    if bbox_array.shape != (480, 640, 4):
        raise ValueError("bbox_array must have the shape (480, 640, 4)")

    # Convert the NumPy array to a PIL image
    img = Image.fromarray(bbox_array, 'RGBA')
    draw = ImageDraw.Draw(img)

    try:
        font_path = '/content/bold.ttf'  # Path where the font is downloaded
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        raise IOError("Font file not found. Ensure the font is downloaded correctly.")

    # Draw the text on the image
    draw.text(text_position, text, fill=(0, 0, 255, 255), font=font)  # Black text

    # Save the image to a BytesIO buffer
    iobuf = io.BytesIO()
    img.save(iobuf, format='png')

    # Encode the image to Base64
    img_bytes = 'data:image/png;base64,{}'.format(
        b64encode(iobuf.getvalue()).decode('utf-8')
    )

    return img_bytes



In [10]:
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0

while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
      break

  img0 = js_to_image(js_reply["img"])


  bbox_array = np.zeros([480,640,4], dtype=np.uint8)



  try:

    objs = DeepFace.analyze(
    img_path = img0,
    actions = ['age', 'gender', 'race', 'emotion'],
  )
    emotion = objs[0]['dominant_emotion']

    base64 = text_to_bytes(bbox_array, text=emotion)
    bbox = base64

  except:

    base64 = text_to_bytes(bbox_array)
    bbox = base64

Action: emotion: 100%|██████████| 4/4 [00:00<00:00, 49.32it/s]


MessageError: ReferenceError: stream_frame is not defined

In [8]:
!wget -q -O /content/bold.ttf https://github.com/googlefonts/roboto/blob/main/src/hinted/Roboto-Bold.ttf?raw=true

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
